Sélection du Modèle d'Embedding

La sélection du modèle d'embedding est une étape cruciale dans tout projet utilisant des
techniques de traitement automatique du langage naturel (NLP), particulièrement lorsqu'il s'agit
d'extraire des informations pertinentes à partir de textes.

# Benchmark des modèles d’embedding

Ce tableau compare **trois modèles d’embeddings** (GloVe, DistilBERT, SentenceTransformers) sur :

- la **similarité moyenne** du Top-30  
- le **temps de traitement**  
- les données : `indeed_offres_data_ai.json` + `profile.json`

---

## ✅ Résultats

| Modèle                  | Similarité Moyenne (Top-30) | Temps de Traitement (sec) |
|-------------------------|-----------------------------|----------------------------|
| **GloVe**               | **0.9859**                  | **0.80**                   |
| **DistilBERT**          | 0.6130                      | 31.60                      |
| **SentenceTransformers**| 0.4595                      | 19.34                      |

---

## 📌 Interprétation

- **GloVe** est le plus rapide et donne une similarité très élevée →  
  

- **DistilBERT** capte beaucoup mieux le sens des phrases →  
  Plus lent mais **plus fiable** pour la vraie sémantique.

- **SentenceTransformers (MiniLM)** est un compromis :  
  **rapide**, moderne, bonne qualité sémantique.




In [15]:
import numpy as np
import pandas as pd
import gensim.downloader as api
import json

In [5]:
# 1) Imports des modèles
# -------------------------
import gensim.downloader as api

# 🔹 Choix des modèles
GLOVE_NAME = "glove-wiki-gigaword-100"            # Glove 

# Chargement de modéle
glove_model = api.load(GLOVE_NAME)   # GloVe (vecteurs de mots)
print("✅ Modèles chargés.")

# -------------------------


[==================================================] 100.0% 128.1/128.1MB downloaded
✅ Modèles chargés.


In [13]:
# 2) Fonctions utilitaires
# -------------------------

JOBS_PATH = "offres_structured.json"
USER_PROFILE_PATH = "user_profile.json"
TOP_K = 30
MAX_JOBS = 300  # pour aller vite


def build_job_text(df: pd.DataFrame) -> pd.Series:
    """title + description + requirements (si dispo)."""
    parts = []
    for col in ["title", "description", "requirements"]:
        if col in df.columns:
            parts.append(df[col].fillna(""))

    if not parts:
        raise ValueError(
            "Aucune des colonnes ['title', 'description', 'requirements'] "
            "n'existe dans le DataFrame."
        )

    full_text = parts[0]
    for p in parts[1:]:
        full_text = full_text + " " + p

    return full_text


def build_user_text(profile: dict) -> str:
    fields = [
        profile.get("target_job", ""),
        profile.get("skills", ""),
        profile.get("experience", ""),
        profile.get("education", ""),
        profile.get("location", ""),
        profile.get("notes", ""),
    ]
    return " ".join([str(f) for f in fields if f])


def encode_glove(texts):
    """
    Embedding moyen de chaque phrase avec GloVe.
    - texts : liste de strings
    - glove : modèle Gensim GloVe
    Retour : np.array (n_texts, dim)
    """
    global glove_model # Use the globally loaded glove_model
    dim = glove_model.vector_size
    embs = []

    for t in texts:
        words = str(t).split()
        vecs = []
        for w in words:
            if w in glove_model:
                vecs.append(glove_model[w])
        if len(vecs) == 0:
            embs.append(np.zeros(dim))
        else:
            embs.append(np.mean(vecs, axis=0))
    return np.vstack(embs)






def cosine_similarity(user_emb: np.ndarray, job_embs: np.ndarray) -> np.ndarray:
    user_norm = user_emb / (np.linalg.norm(user_emb) + 1e-8)
    jobs_norm = job_embs / (np.linalg.norm(job_embs, axis=1, keepdims=True) + 1e-8)
    scores = jobs_norm @ user_norm
    return scores


def top_k_indices(scores: np.ndarray, k: int = 20) -> np.ndarray:
    k = min(k, len(scores))
    idx = np.argpartition(-scores, k - 1)[:k]
    idx_sorted = idx[np.argsort(-scores[idx])]
    return idx_sorted


# Dictionnaire : nom du modèle → fonction d'encodage
EMBEDDING_MODELS = {
    "GloVe": encode_glove,
    
}

In [14]:

# 1) Charger le modèle GloVe (une seule fois)
print("Chargement du modèle GloVe...")
glove_model = api.load("glove-wiki-gigaword-300")  # 300 dimensions
print("GloVe chargé :", glove_model.vector_size, "dimensions")

def main_global_profile():
    # 2) Charger les offres de travail
    df_jobs = pd.read_json(JOBS_PATH)

    # Optionnel : limiter pour aller vite
    df_jobs = df_jobs.head(MAX_JOBS).copy()

    # 3) Construire le texte des offres
    job_texts = build_job_text(df_jobs)
    # job_texts est une Series, on la convertit en liste
    job_texts_list = job_texts.tolist()

    # 4) Encoder les offres avec GloVe
    job_embs = encode_glove(job_texts_list)  # shape (n_jobs, dim)

    # 5) Charger le profil utilisateur
    with open(USER_PROFILE_PATH, "r", encoding="utf-8") as f:
        user_profile = json.load(f)

    # 6) Construire le texte global du profil
    user_text = build_user_text(user_profile)

    # 7) Encoder le profil (une seule phrase => [0, :])
    user_emb = encode_glove([user_text])[0]  # shape (dim,)

    # 8) Calculer les similarités profil ↔ offres
    scores = cosine_similarity(user_emb, job_embs)  # shape (n_jobs,)

    # 9) Récupérer les meilleurs TOP_K offres
    idx_top = top_k_indices(scores, k=TOP_K)

    # 10) Afficher les résultats
    print("\n=== TOP offres GloVe pour ce profil (global) ===")
    for rank, idx in enumerate(idx_top, start=1):
        row = df_jobs.iloc[idx]
        print(f"\n#{rank}  (score={scores[idx]:.4f})")
        print("Titre :", row.get("title", "N/A"))
        print("Entreprise :", row.get("company", "N/A"))
        print("Lieu :", row.get("location", "N/A"))
        # tu peux afficher un résumé de la description :
        desc = str(row.get("description", ""))[:200].replace("\n", " ")
        print("Description :", desc, "...")
        

if __name__ == "__main__":
    main_global_profile()


Chargement du modèle GloVe...
GloVe chargé : 300 dimensions

=== TOP offres GloVe pour ce profil (global) ===

#1  (score=0.8440)
Titre : Stage PFE : IA / NLP — Fine-Tuning à partir de PDFs
Entreprise : Koding Schools
Lieu : Remote
Description :  ...

#2  (score=0.8440)
Titre : Stage PFE : IA / NLP — Fine-Tuning à partir de PDFs
Entreprise : Koding Schools
Lieu : Remote
Description :  ...

#3  (score=0.8440)
Titre : Stage PFE : IA / NLP — Fine-Tuning à partir de PDFs
Entreprise : Koding Schools
Lieu : Remote
Description :  ...

#4  (score=0.7473)
Titre : Offre de Stage – Développeur en Intelligence Artificielle (IA)
Entreprise : AI HUB
Lieu : Rabat
Description :  ...

#5  (score=0.6626)
Titre : Data Scientist / Ingénieur IA Confirmé (3 à 6 ans d’expérience)
Entreprise : AdmitEase
Lieu : Casablanca
Description :  ...

#6  (score=0.6626)
Titre : Data Scientist / Ingénieur IA Confirmé (3 à 6 ans d’expérience)
Entreprise : AdmitEase
Lieu : Casablanca
Description :  ...

#7  (score=0.6071)
